# Stock_Query
This script runs the output of the Query_List.ipynb (Query_List.csv) and returns 20 years of daily-time series data for each stock. Data is pulled from the Alpha Vantage API. Data is then to be used in analytica for research.

Script PFD:

1. FortuneList Sorting 
2. Ticker_Lookup(in FortuneList) 
3. Symbol_Corrections(in FortuneList) 
4. Stock_Query 
5. StockMarketCrash_Analytica

In [9]:
# importing dependencies
import time
import pandas as pd
from config import api_key_av
import requests


# reading in query_list for required tickers
query_list = pd.read_csv('../Resources/Query_List.csv')
query_list['ticker'].tolist()
tickers = ['ARNC', 'MSFT', 'KO', 'ODP', 'OMX', 'GE']

# terminal print out of script performance
print(f'''
!Query Initiated!
-----------------
''')


# creating hard-coded values (for ease of future expansion)
max_call = 5 #calls per minute allowed by Alpha Vantage
wait_time = 60

# creating loop counters & summary stat counters
success = 0 #success API call counter
failed = 0 #failed API call counter
pull_time = 0 #future pull time variable
counter = 0 #API call counter

# creating master dataframe (to append to)
master_df = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker'])

# primary loop for ticker call list to API
for record, ticker in enumerate(tickers):
    
    # setting API call parameters (ticker)
    params = {
        'function' : 'TIME_SERIES_DAILY',
        'symbol' : ticker,
        'outputsize' : 'full',
        'datatype' : 'json',
        'apikey' : api_key_av
    }
    
    # base API call URL
    url = 'https://www.alphavantage.co/query?'
    
    # updating time to now
    now = time.time()
    
    # API call wait function (limited to 5 calls per minute); is utilized after counter resets from 5 to 0
    if counter == 0:
        while now < pull_time:
            now = time.time()
    
    # if statement verifying proper time and counter for being able to make a successful API call
    if pull_time < now and counter < max_call:
        
        # trying API call and skipping if failed for any reason
        try:
            
            # making API call and storing data into variable
            response = requests.get(url, params).json()
                                    
            # THE MEAT AND POTATOS
            # creating main ticker dataframe
            ticker_df = pd.DataFrame(response['Time Series (Daily)']).transpose().reset_index()
            
            # renaming columns to better indicators
            ticker_df = ticker_df.rename(columns={
                'index':'Date',
                '1. open':'Open',
                '2. high':'High',
                '3. low':'Low',
                '4. close':'Close',
                '5. volume':'Volume'
            })
            
            # pulling ticker symbol for confirmation purposes from API
            ticker_df['Ticker'] = response['Meta Data']['2. Symbol']
            
            
            # appending items to master dataframe
            master_df = master_df.append(ticker_df)
            
            row_count = ticker_df['Date'].count()
            
            print(f'Record {record + 1} out of {len(tickers)} processed. {ticker} : SUCCESS({row_count} rows)')
            
            success += 1
            counter += 1
            
            # resetting counter to 0 to initiate next call limit & wait function
            if counter == max_call:
                pull_time = int(time.time()) + wait_time
                counter = 0
                
        # handeling failed API calls
        except:
            print(f'Record {record + 1} out of {len(tickers)} processed. {ticker} : FAILED')
            print(response)
            failed += 1


# writing master dataframe to csv, indicating now timestamp to keep from overwritting accidentally
master_df.to_csv(f'../Resources/Master_Datasets/master_dataset_{int(now)}.csv', index=False)


# terminal print out of script performance
print(f'''
DONE
------------------
Query Summary
Success: {success} 
Failed: {failed}
Total: {record + 1}
''')


!Query Initiated!
-----------------

Record 1 out of 6 processed. ARNC : SUCCESS(5034 rows)
Record 2 out of 6 processed. MSFT : SUCCESS(5034 rows)
Record 3 out of 6 processed. KO : SUCCESS(5034 rows)
Record 4 out of 6 processed. ODP : SUCCESS(5034 rows)
Record 5 out of 6 processed. OMX : SUCCESS(1726 rows)
Record 6 out of 6 processed. GE : SUCCESS(5034 rows)

DONE
------------------
Query Summary
Success: 6 
Failed: 0
Total: 6



In [6]:
# importing dependencies
import time
import pandas as pd
from config import api_key_av
import requests


# reading in query_list for required tickers
query_list = pd.read_csv('../Resources/Query_List.csv')

tickers = query_list['ticker'].tolist()
# ['ARNC', 'MSFT', 'KO', 'ODP', 'OMX', 'GE']

# terminal print out of script performance
print(f'''
!Query Initiated!
-----------------
''')

# creating limit response note from AV
bad_call = 'Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.'


# creating loop counters & summary stat counters
success = 0 #success API call counter
failed = 0 #failed API call counter

# creating master dataframe (to append to)
master_df = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker'])

# primary loop for ticker call list to API
for record, ticker in enumerate(tickers):
    
    # setting API call parameters (ticker)
    params = {
        'function' : 'TIME_SERIES_DAILY',
        'symbol' : ticker,
        'outputsize' : 'full',
        'datatype' : 'json',
        'apikey' : api_key_av
    }
    
    # base API call URL
    url = 'https://www.alphavantage.co/query?'


    # trying API call and skipping if failed for any reason
    try:
        
        # making API call and storing data into variable
        response = requests.get(url, params).json()
        
        try:
            if response['Note'] == bad_call:
                while response['Note'] == bad_call:
                    response = requests.get(url, params).json()

        except:

            # THE MEAT AND POTATOS
            # creating main ticker dataframe
            ticker_df = pd.DataFrame(response['Time Series (Daily)']).transpose().reset_index()

            # renaming columns to better indicators
            ticker_df = ticker_df.rename(columns={
                'index':'Date',
                '1. open':'Open',
                '2. high':'High',
                '3. low':'Low',
                '4. close':'Close',
                '5. volume':'Volume'
            })

            # pulling ticker symbol for confirmation purposes from API
            ticker_df['Ticker'] = response['Meta Data']['2. Symbol']


            # appending items to master dataframe
            master_df = master_df.append(ticker_df)

            row_count = ticker_df['Date'].count()

            print(f'Record {record + 1} out of {len(tickers)} processed. {ticker} : SUCCESS({row_count} rows)')

            success += 1


    # handeling failed API calls
    except:
        print(f'Record {record + 1} out of {len(tickers)} processed. {ticker} : FAILED')

        failed += 1


# writing master dataframe to csv, indicating now timestamp to keep from overwritting accidentally
master_df.to_csv(f'../Resources/Master_Datasets/master_dataset_{int(now)}.csv', index=False)


# terminal print out of script performance
print(f'''
DONE
------------------
Query Summary
Success: {success} 
Failed: {failed}
Total: {record + 1}
''')


!Query Initiated!
-----------------

Record 1 out of 164 processed. XOM : SUCCESS(5034 rows)
Record 2 out of 164 processed. BRK-A : SUCCESS(5034 rows)
Record 3 out of 164 processed. T : SUCCESS(5034 rows)
Record 4 out of 164 processed. C : SUCCESS(5034 rows)
Record 5 out of 164 processed. MSFT : SUCCESS(5034 rows)
Record 6 out of 164 processed. WFC : SUCCESS(5034 rows)
Record 7 out of 164 processed. GE : SUCCESS(5034 rows)
Record 8 out of 164 processed. F : SUCCESS(5034 rows)
Record 9 out of 164 processed. PFE : SUCCESS(5034 rows)
Record 10 out of 164 processed. INTC : SUCCESS(5034 rows)
Record 11 out of 164 processed. JNJ : SUCCESS(5034 rows)
Record 12 out of 164 processed. COP : SUCCESS(5034 rows)
Record 13 out of 164 processed. PG : SUCCESS(5034 rows)
Record 14 out of 164 processed. MO : SUCCESS(5034 rows)
Record 15 out of 164 processed. MRK : SUCCESS(5034 rows)
Record 16 out of 164 processed. PEP : SUCCESS(5034 rows)
Record 17 out of 164 processed. UNH : SUCCESS(5034 rows)
Record 

Record 145 out of 164 processed. R : SUCCESS(5034 rows)
Record 146 out of 164 processed. nan : SUCCESS(5034 rows)
Record 147 out of 164 processed. JWN : SUCCESS(5034 rows)
Record 148 out of 164 processed. ARW : SUCCESS(5034 rows)
Record 149 out of 164 processed. AVT : SUCCESS(5034 rows)
Record 150 out of 164 processed. FL : SUCCESS(5034 rows)
Record 151 out of 164 processed. CMS : SUCCESS(5034 rows)
Record 152 out of 164 processed. SVU : SUCCESS(4776 rows)
Record 153 out of 164 processed. PBI : SUCCESS(5034 rows)
Record 154 out of 164 processed. CCK : SUCCESS(5034 rows)
Record 155 out of 164 processed. NCR : SUCCESS(5034 rows)
Record 156 out of 164 processed. ODP : SUCCESS(5034 rows)
Record 157 out of 164 processed. NWL : SUCCESS(5034 rows)
Record 158 out of 164 processed. BLL : SUCCESS(5034 rows)
Record 159 out of 164 processed. AVY : SUCCESS(5034 rows)
Record 160 out of 164 processed. OMX : SUCCESS(1726 rows)
Record 161 out of 164 processed. RRD : SUCCESS(5034 rows)
Record 162 out of

In [7]:
# reviewing last master_dataset output
lmds_df = pd.read_csv(f'../Resources/Master_Datasets/master_dataset_{int(now)}.csv')
lmds_df

,Date,Open,High,Low,Close,Volume,Ticker
0,2019-12-20,69.86,70.2267,69.39,69.94,35091987,XOM
1,2019-12-19,69.99,70.0500,69.24,69.39,13273228,XOM
2,2019-12-18,69.63,70.2900,69.28,69.87,13652100,XOM
3,2019-12-17,69.98,70.3800,69.31,69.68,14450050,XOM
4,2019-12-16,69.70,70.2500,69.70,70.00,15208665,XOM
...,...,...,...,...,...,...,...
770348,1999-12-27,8.44,8.7500,8.25,8.50,62800,OMI
770349,1999-12-23,8.62,9.0000,8.50,8.56,45267,OMI
770350,1999-12-22,8.13,8.5000,8.06,8.50,259400,OMI
770351,1999-12-21,8.37,8.3700,8.06,8.31,102400,OMI


In [8]:
'''
response:
{'Note': 'Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 
500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.'}
'''

"\nfor making it fast:\n\n\nif response['Note'] = bad_call:\n    while response['Note'] = bad_call:\n        response = requests.get(url, params).json()\n\nbad_call = 'Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.'\n\n\n\nalso remove the wait loops\n\nresponse:\n{'Note': 'Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and \n500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.'}\n"

In [ ]:
'''
recommendation / hybrid:

1open working csv to df to append to
2spam API until get bad_call response
3save off data to dataframe to csv
4wait using system time until next minute i.e. ((time.time)/60).isinteger


'''